In [13]:
import pandas as pd
import nltk
import spacy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score

In [14]:
import pandas as pd

# Cargar los datos de entrenamiento y prueba
train_path = "FinancES_train_kaggle.csv"  # Ruta del archivo de entrenamiento
test_path = "FinancES_test_kaggle.csv"    # Ruta del archivo de prueba

# Leer los archivos CSV
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Mostrar información de los datos cargados
print("Train Dataset:")
print(train_df.info())
print(train_df.head())

print("\nTest Dataset:")
print(test_df.info())
print(test_df.head())

Train Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6359 non-null   int64 
 1   text    6359 non-null   object
 2   label   6359 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 149.2+ KB
None
   id                                               text  label
0   0  Renfe afronta mañana un nuevo día de paros par...      2
1   1       Presupuesto populista con cimientos frágiles      2
2   2  Biden no cree que la OPEP+ vaya a ayudar con l...      2
3   3  La deuda de las familias cae en 25.000 millone...      0
4   4  Bestinver: no hay "momento más inoportuno" par...      2

Test Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1621 non-null   int64 
 1   text    1621 non-null   

In [15]:
# Exploracion

# Identificar distribución de las clases
print("\nDistribución de las clases en el conjunto de entrenamiento:")
print(train_df["label"].value_counts())

# Verificar si hay valores nulos
print("\nValores nulos en el conjunto de entrenamiento:")
print(train_df.isnull().sum())

print("\nValores nulos en el conjunto de prueba:")
print(test_df.isnull().sum())



Distribución de las clases en el conjunto de entrenamiento:
label
2    2935
0    2818
1     606
Name: count, dtype: int64

Valores nulos en el conjunto de entrenamiento:
id       0
text     0
label    0
dtype: int64

Valores nulos en el conjunto de prueba:
id      0
text    0
dtype: int64


In [16]:
# Análisis de longitud de los textos
train_df["text_length"] = train_df["text"].apply(len)

print("\nEstadísticas de la longitud de los textos en el conjunto de entrenamiento:")
print(train_df["text_length"].describe())


Estadísticas de la longitud de los textos en el conjunto de entrenamiento:
count    6359.000000
mean       72.736751
std        20.955372
min         3.000000
25%        61.000000
50%        72.000000
75%        84.000000
max       211.000000
Name: text_length, dtype: float64


In [17]:
import re

# Normalización: convertir a minúsculas
train_df["text"] = train_df["text"].str.lower()
test_df["text"] = test_df["text"].str.lower()

print("\nTexto preprocesado:")
print(train_df["text"].head())



Texto preprocesado:
0    renfe afronta mañana un nuevo día de paros par...
1         presupuesto populista con cimientos frágiles
2    biden no cree que la opep+ vaya a ayudar con l...
3    la deuda de las familias cae en 25.000 millone...
4    bestinver: no hay "momento más inoportuno" par...
Name: text, dtype: object


In [18]:
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

# Cargar los datos de entrenamiento y prueba
train_path = "FinancES_train_kaggle.csv"  # Ruta del archivo de entrenamiento
test_path = "FinancES_test_kaggle.csv"    # Ruta del archivo de prueba

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
import spacy
from collections import Counter
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Cargar modelo de spaCy para español (mejor que stemming para Word2Vec)
nlp = spacy.load("es_core_news_sm")

# Contar palabras más frecuentes en el dataset
all_words = " ".join(train_df["text"]).split()
word_freq = Counter(all_words)

# Eliminar palabras que aparecen en más del 80% de los documentos
frequent_words = {word for word, freq in word_freq.items() if freq > 0.8 * len(train_df)}

# Lista de stopwords personalizada
custom_stopwords = set(stopwords.words('spanish'))

# Agregar palabras irrelevantes para titulares financieros
custom_stopwords.update({"día", "años", "mes", "nuevo", "euros"})

# Añadir palabras frecuentes a la lista de stopwords
custom_stopwords.update(frequent_words)

# Función de preprocesamiento con Lematización y stopwords mejoradas
def preprocess_text(text):
    # Tokenización con spaCy
    doc = nlp(text.lower())  # Convertir a minúsculas y procesar con spaCy

    # Eliminar stopwords y obtener lemas
    tokens = [token.lemma_ for token in doc if token.text not in custom_stopwords and not token.is_punct]

    return " ".join(tokens)

# Aplicar preprocesamiento
train_df["text"] = train_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)

In [21]:
import numpy as np

def text_to_vector(text, model):
    words = word_tokenize(text)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  # Promedio de los vectores de las palabras
    else:
        return np.zeros(model.vector_size)  # Vector de ceros si el texto no tiene palabras conocidas

# Convertir todos los textos a vectores
X_train = np.array([text_to_vector(text, word2vec_model) for text in train_df["text"]])
X_test = np.array([text_to_vector(text, word2vec_model) for text in test_df["text"]])
y_train = train_df["label"]


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Aplicar validación cruzada con los hiperparámetros encontrados
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=2, random_state=42)

scores = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='f1_macro')

print("F1-Score por Fold:", scores)
print("F1-Score promedio:", scores.mean())

# Entrenar el modelo con los datos completos
rf_model.fit(X_train, y_train)

# Generar predicciones para el conjunto de test
test_predictions = rf_model.predict(X_test)
test_df["label"] = test_predictions


F1-Score por Fold: [0.38055917 0.37926724 0.38706388 0.39192533 0.39188714]
F1-Score promedio: 0.38614055452637286


**NO CREO QUE FUNCIONE PERO EN CUALQUIER CASO, LO QUE CREO QUE ES LO ÚNICO QUE PODRÍA HACER FUNCIONAR A ESTO ES COGER UNOS EMBEDDINGS PREENTRENADOS [LINK A WORD EMBEDDINGS EN ESPAÑOL DE GITHUB](https://github.com/dccuchile/spanish-word-embeddings)**